In [2]:
import pandas as pd
import json
import torch
import sys
import os
import re
import time
import gc
import json
from PIL import Image
from transformers import MllamaForConditionalGeneration, AutoProcessor
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
MSG_PRINTED = False
DF_PROMPT = (
    "You are an expert in disaster response and humanitarian aid data analysis. "
    "Examine the given text and image carefully and classify them into exactly one of these categories (0-4). "
    "Respond with ONLY the number, no other text or explanation.\n\n"
    "Categories:\n"
    "0: HUMAN IMPACT - Must be about PEOPLE who are clearly affected by the disaster: injured, displaced, "
    "evacuated, in temporary shelters, or waiting in lines for aid. People must be visibly impacted.\n"
    
    "1: RESPONSE EFFORTS - Must be about active RESCUE operations, aid distribution, medical treatment, "
    "VOLUNTEER work, DONATION, or evacuation in progress. Look for emergency responders, relief workers, or "
    "organized aid activities.\n"
    
    "2: INFRASTRUCTURE DAMAGE - Must be about clear physical damage to buildings, roads, bridges, power lines, "
    "VEHICLES or other structures that was caused by the disaster. The damage should be obvious and significant.\n"
    
    "3: OTHER CRISIS INFO - Must be about verified crisis-related content that doesn't fit above categories: "
    "maps of affected areas, emergency warnings, official updates, or documentation of crisis conditions. "
    "Must have clear connection to the current disaster.\n"
    
    "4: NOT CRISIS-RELATED - Use this for:\n"
    "- Images and text where you're unsure if they are related to the crisis\n"
    "- General texts and photos that could be from any time/place\n" 
    "- Texts and images without clear crisis impact or response\n"
    "- Texts are not related to a crisis with stock photos or promotional images\n"
    "- Any text and image that doesn't definitively fit categories 0-3\n\n"
    
    "Important:\n"
    "- If there's ANY sign of rescue or donation, pick 1.\n"
    "- If there's ANY sign of damage, pick 2.\n"
    "- If there's ANY sign of obviously distressed or harmed people, pick 0.\n"
    "- If the text and image are definitely about a crisis but you DO NOT see rescue/damage/impacted people, pick 3.\n"
    "- Otherwise, pick 4. Also, when you are not sure which number to pick, pick 4.\n"
    "Answer with just a single digit (0–4)."
)

INFO_TEXT_ONLY_PROMPT = """ You are an AI model that classifies the given text into one of two categories based on its informational value. 
Your task is to analyze the given text to determine whether it contains relevant or informative content about a crisis.
Our goal is to collect as much useful information as possible about crises, so your classification should prioritize identifying texts that provide relevant details, even if they are brief or incomplete. 
Avoid being overly restrictive. If the text has any relevant crisis-related information, classify it as informative.
Classify the text delimited by triple quotes (\"\"\" \"\"\") into one of the following categories:
  - **1 (positive):** The text provide details, updates, or any relevant information about a crisis.
  - **0 (not positive):** The text do not contain relevant details or information about a crisis.
Return only the classification label (1 or 0) without any extra text or explanation.

"""   

INFO_IMAGE_ONLY_PROMPT = """Does the image give useful information that could help during a crisis?
Respond with '1' if this image provides any information or details about a crisis, and '0' if it does not.

Instructions: 
  - You should prioritize identifying texts that provide relevant details, even if they are brief or incomplete.
  - Avoid being overly restrictive. If the text has any relevant crisis-related information, response with '1'.
  - When the meaning of the image is unclear, response with '0'.
  - Do not output any extra text.
"""

INFO_TEXT_IMAGE_PROMPT = """Do the given text and the given image give useful information that could help during a crisis?
Respond with '1' if the text and the image provide any information or details about a crisis, and '0' if they do not.

Instructions: 
  - You should prioritize identifying texts and the images that provide relevant details, even if they are brief or incomplete.
  - Avoid being overly restrictive. If the text and image have any relevant crisis-related information, response with '1'.
  - When the meaning of the image and the text are unclear, response with '0'.
  - Do not output any extra text.
"""

# Function to detect the match patterns in the model output
def detect_informative_label(text):
    # Check for "not provide any information" first to avoid partial matches
    if re.search(r"\bnot (provide |give |contain |related |relevant )?any information\b", text, re.IGNORECASE) or re.search(r"\b0[.!]?\b", text):
        print("Analyze successfully! This entry is not informative!")
        return 0
    elif re.search(r"\b(provides |provide |gives |give )(useful |some |relevant )?information\b", text, re.IGNORECASE)  or re.search(r"\b1[.!]?\b", text):
        print("Analyze successfully! This entry is informative!")
        return 1
    print("Analyze completed! No matching label found! Assigned as not informative by default!") 
    return 0  # Return 0 if neither phrase is found

def detect_humanitarian_label(text):
    # Clean all symbols
    text = re.sub(r'[^A-Za-z0-9\s]', '', text)
    
    # First, check for "not provide any information" or the digit "4"
    if re.search(r"\b(not (provide|give|contain)|4|\*\*4\*\*)\b", text, re.IGNORECASE):
        print("  Analyzed successfully! This entry is 4: not humanitarian!")
        return 4

    # Check for case "0": Ensure the text contains "people" and one of "impact", "impacted", or "affected", and "crisis"
    elif (
        re.search(r"\bpeople\b", text, re.IGNORECASE) and re.search(r"\bcrisis\b", text, re.IGNORECASE)
    ) or (
        re.search(r'(?=.*\banswer|response|number\b)(?=.*\b0\b)', text, re.IGNORECASE)
    ) or (
        re.search(r"\b\*\*0\*\*\b", text, re.IGNORECASE)
    ):
        print("  Analyzed successfully! This entry is 0: affected individuals!")
        return 0

    # Check for rescue, volunteering, donation efforts, or the digit "1"
    elif re.search(r"\b(rescue|volunteering|donation|1|\*\*1\*\*)\b", text, re.IGNORECASE):
        print("  Analyzed successfully! This entry is 1: rescue, volunteering, or donation efforts!")
        return 1

    # Check for infrastructure, buildings, utilities, vehicles, or the digit "2"
    elif re.search(r"\b(infrastructure|building|utility|vehicle|car|2|\*\*2\*\*)\b", text, re.IGNORECASE):
        print("  Analyzed successfully! This entry is 2: infrastructure and utility damage!")
        return 2

    # Check for "other" category or the digit "3"
    elif re.search(r"\b(other|3|\*\*3\*\*)\b", text, re.IGNORECASE):
        print("  Analyzed successfully! This entry is 3: other relevant information!")
        return 3

    # Default case: No match found
    print("  Analyze completed! No matching label found! Assigned as 4: not humanitarian by default!")
    return 4  # Default to 4 if no label matches

def load_few_shot_examples(devdata_path, few_shot_num):
    """Loads few-shot examples with unique labels from the dev set.
    
    If few_shot_num=1, it returns an array where each entry is randomly selected from a different label.
    If few_shot_num > 1, it returns `few_shot_num` examples per label (if available).
    """
    if not os.path.isfile(devdata_path):
        raise FileNotFoundError(f"Dev file not found: {devdata_path}")
    
    if devdata_path.endswith(".json"):
        dev_data = pd.read_json(devdata_path, lines=False, encoding="utf-8")
    elif devdata_path.endswith(".csv"):
        dev_data = pd.read_csv(devdata_path, encoding="utf-8-sig")
    else:
        raise ValueError("Dev dataset must be .json or .csv")
    
    required_columns = ['text', 'image_path', 'label']
    if not all(column in dev_data.columns for column in required_columns):
        raise ValueError("Dev data must contain columns: text, image_path, label")
    
    # Select `few_shot_num` random examples per label
    label_samples = dev_data.groupby("label").apply(lambda x: x.sample(n=min(few_shot_num, len(x)), random_state=42)).reset_index(drop=True)

    return label_samples.to_dict(orient="records")

def load_and_process_image(image_path):
    """Loads and processes an image (returns a PIL Image) for the model."""
    assert os.path.isfile(image_path), f"File not found: {image_path}"
    return Image.open(image_path).convert("RGB")


def construct_message_template(few_shot_examples, prompt, test_text=None, test_image=None, use_texts=True, use_images=True, few_shot_num=0):
    """
    Constructs the structured message format with both few-shot examples and test case.
    
    Example (Few-shot case):
    [
      {"role": "user", "content": [{"type": "text", "text": "<Prompt> Here are sample data with labels:"}]}, 
      {"role": "user", "content": [{"type": "image", "image": sample_image_1}, {"type": "text", "text": sample_text_1}]},
      {"role": "assistant", "content": label_1},
      ....
      {"role": "user", "content": [{"type": "text", "text": "Now classify this test data:"}, {"type": "image", "image": test_image}]}
    ]

    Example (Zero-shot case):
    [
      {"role": "user", "content": [{"type": "text", "text": "<Prompt>"}]},
      {"role": "user", "content": [{"type": "text", "text": "Now classify this test image:"}, {"type": "image", "image": test_image}]}
    ]
    """
    """messages = [
        {
            "role": "system",
            "content": [
                {"type": "image"},
                {"type": "text", "text": f"{prompt}.\nHere are some examples:\n"}
            ],
        }
    ]"""

    messages = []

    def ordinal(n):
        """Convert an integer to its ordinal representation (e.g., 2 -> 2nd, 3 -> 3rd, 4 -> 4th, 11 -> 11th)."""
        if 10 <= n % 100 <= 20:  # Special case for 11th, 12th, 13th, etc.
            suffix = "th"
        else:
            suffix = {1: "st", 2: "nd", 3: "rd"}.get(n % 10, "th")
        return f"{n}{suffix}"

    user_content = []
    # Add few-shot examples if available
    if few_shot_num > 0 and few_shot_examples:          
        for index, example in enumerate(few_shot_examples):
            #user_content = []
            txt_prompt = "Below is an example for your reference:\n"
            if use_texts and use_images: # text + image
                user_content.append({"type": "image", "image": example["image"]})
                #user_content.append({"type": "text", "text": f"\nText: \"\"\"{example['text']}\"\"\"\nThe category of this sample text and image is: "})
                txt_prompt = f"Above is the image, and here is the Text: {example['text']}\nThe category of this image and Text is: "
            elif use_texts: # text only
                #user_content.append({"type": "text", "text": f"\n{prompt}\nText: \"\"\"{example['text']}\"\"\"\nThe category of this sample text is: "})   
                txt_prompt = f"Text: \"\"\"{example['text']}\"\"\"\nThe category of this sample text is: "
            else: # image only
                user_content.append({"type": "image", "image": example["image"]})
                #user_content.append({"type": "text", "text": f"\n{prompt}\nThe category of this sample image is: "})
                txt_prompt = f"Above is the sample image. The category of this sample image is: "
            txt_prompt += str(example['label'])
            user_content.append({"type": "text", "text": txt_prompt})    
            #messages.append({"role": "user", "content": user_content})
            #messages.append({"role": "assistant", "content": str(example['label'])})

    #user_content = []
    txt_prompt = ""

    # Adding the test data to the message
    if use_texts and use_images:
        user_content.append({"type": "image", "image": test_image})
        #user_content.append({"type": "text", "text": f"\n{prompt}\nText: \"\"\"{test_text}\"\"\"\n\nThe category of this Text and the above image is: "})
        txt_prompt = f"{prompt}\nAbove is the given image, and here is the given Text: {test_text}.\nThe category of this image and Text is: "
    elif use_texts:
        #user_content.append({"type": "text", "text": f"\n{prompt}\nText: \"\"\"{test_text}\"\"\"\n\nThe category of this test Text is:"}) 
        txt_prompt = f"{prompt}\nHere is the test Text that you need to classify: \"\"\"{test_text}\"\"\". The category of this test Text is: "
    else:
        user_content.append({"type": "image", "image": test_image})
        #user_content.append({"type": "text", "text": f"\n{prompt}\n\nThe category of this image is:"})     
        txt_prompt = f"{prompt}\nAbove is the test image that you need to classify. The category of this test image is: "

    user_content.append({"type": "text", "text": txt_prompt})    
    messages.append({"role": "user", "content": user_content})             

    #print("\n\n", messages, "\n\n")
    return messages

def classify_raw_text(raw_text, task="Informative"):
    # Basic parsing: split on the word "assistant" to isolate answer
    parts = raw_text.split("assistant")
    
    if len(parts) > 1:
        final_answer = parts[-1].strip()
        if task=="Informative":            
            if final_answer.startswith("1"):
                return 1
            elif final_answer.startswith("0"):
                return 0
            else:
                # If it doesn't strictly start with '1' or '0', handle as needed
                print(f"Unclear model output: {final_answer}! Trying to analyze the output again...")
                # Trying to look for expected output patterns
                return detect_informative_label(final_answer)
        else: # Humanitarian
            if final_answer.startswith("0"):
                return 0
            elif final_answer.startswith("1"):
                return 1
            elif final_answer.startswith("2"):
                return 2   
            elif final_answer.startswith("3"):
                return 3
            elif final_answer.startswith("4"):
                return 4             
            else:
                # If it doesn't strictly start with '1' or '0', handle as needed
                print(f"\nUnclear model output: {final_answer}!\n  Trying to analyze the output again...")
                # Trying to look for expected output patterns
                return detect_humanitarian_label(final_answer)
    else:
        print(f"Could not parse output: {raw_text}")
        return None
    
def predict(model, processor, prompt, task="Informative", 
            use_texts=True, use_images=True, 
            text=None, image_path=None, 
            few_shot_examples=None, few_shot_num=0):
    """
    Predicts using the LLaMA Vision model with properly structured messages.
    Handles text-only, image-only, and text+image few-shot scenarios.
    """
    global MSG_PRINTED

    # Load test image if available
    test_image = load_and_process_image(image_path) if image_path else None
    all_images = []

    # Ensure few-shot examples exist before accessing
    if use_images: # test with image(s)        
        if few_shot_num > 0 and few_shot_examples: # not zeroshot
            for example in few_shot_examples:
                all_images.append(example["image"])       
            if test_image:
                all_images.append(test_image)  # Ensure test image is included    

        elif test_image: # zeroshot with valid image
            all_images = test_image
        else:
            all_images = None
        
    """
    if few_shot_examples and use_images:
        all_images = [example["image"] for example in few_shot_examples if "image" in example]
    else:
        all_images = []

    if test_image:
        all_images.append(test_image)  # Ensure test image is included"""

    # Construct the structured message
    
    messages = construct_message_template(
        few_shot_examples=few_shot_examples,
        prompt=prompt,
        test_text=text,
        test_image=test_image,
        use_texts=use_texts,
        use_images=use_images,
        few_shot_num=few_shot_num
    )

    if not MSG_PRINTED:
        # Function to serialize images only for printing
        def json_serializable(obj):
            if isinstance(obj, Image.Image):
                return f"<PIL.Image mode={obj.mode} size={obj.size}>"
            raise TypeError(f"Type {type(obj)} is not JSON serializable")
        print("\n\n", '-'*80)
        # Print formatted JSON
        print(json.dumps(messages, indent=2, ensure_ascii=False, default=json_serializable))
        print('-'*80, "\n\n")
        MSG_PRINTED = True

    # Apply chat template to format the input
    formatted_input = processor.apply_chat_template(messages, add_generation_prompt=True)

    # Prepare input tensors
    if use_images:
        inputs = processor(
            text=formatted_input,
            images=all_images,
            return_tensors="pt",
            truncation=False,
            #max_length=20480
        ).to(model.device)
    else:
        inputs = processor(
            text=formatted_input,            
            return_tensors="pt",
            truncation=False,
            #max_length=20480
        ).to(model.device)

    # Generate prediction
    with torch.no_grad():
        output = model.generate(
            **inputs,
            max_new_tokens=20,
            min_new_tokens=1,
            do_sample=False,    # Deterministic generation, temperature ignored
            num_beams=1,        # Simple greedy decoding
            pad_token_id=processor.tokenizer.pad_token_id,
            eos_token_id=processor.tokenizer.eos_token_id
        )

    # Decode model output
    raw_text = processor.decode(output[0], skip_special_tokens=True, clean_up_tokenization_spaces=True)

    #print(f"\n\n raw_text = {raw_text}\n\n")
    
    return classify_raw_text(raw_text, task)



def update_remaining_time(start_time, completed_tests, total_tests, checkpoint_label):
    """
    Calculates and prints the estimated remaining time.
    
    Parameters:
    - start_time (float): The timestamp when the tests started.
    - completed_tests (int): Number of completed tests.
    - total_tests (int): Total number of tests.
    - checkpoint_label (str): A message indicating the progress percentage.
    """
    elapsed_time = time.time() - start_time
    avg_time_per_test = elapsed_time / completed_tests
    remaining_time = avg_time_per_test * (total_tests - completed_tests)

    print(f"\n📊 {checkpoint_label} completed. Estimated remaining time: {remaining_time / 60:.2f} minutes.\n")

def test_model(
    model_id="meta-llama/Llama-3.2-11B-Vision-Instruct",
    task="Informative",
    use_texts=True,
    use_images=True,
    prompt=DF_PROMPT,
    testdata_path="dataset/Informative/info_text_image_test.json",
    devdata_path="dataset/Informative/info_text_image_dev.json",
    result_path="test_results/informative/Llama-3.2-11B-Zeroshot.json",
    few_shot_num=0,
    consistency=False
):
    """
    Loads the model, predicts on test data, and saves results.
    
    Handles:
    - Zero-shot, few-shot (if few_shot_num > 0)
    - Text-only, image-only, text+image settings
    - Structured input with user-assistant messages
    """

    testdata_path = os.path.normpath(testdata_path)
    result_path = os.path.normpath(result_path)
    
    print(f"Model={model_id}, testdata={testdata_path}, result={result_path}\n")

    if not os.path.isfile(testdata_path):
        raise FileNotFoundError(f"Test file not found: {testdata_path}")
    
    if result_path:
        result_dir = os.path.dirname(result_path)
        if not os.path.exists(result_dir):
            os.makedirs(result_dir)

    print(f"Loading Llama-Vision model from {model_id} ...")
    model = MllamaForConditionalGeneration.from_pretrained(
        model_id,
        device_map="auto",
        torch_dtype=torch.bfloat16
    )
    processor = AutoProcessor.from_pretrained(model_id)
    model.eval()

    print(f"Loading test dataset from {testdata_path}")
    if testdata_path.endswith(".json"):
        test_data = pd.read_json(testdata_path, lines=False, encoding="utf-8")
    elif testdata_path.endswith(".csv"):
        test_data = pd.read_csv(testdata_path, encoding="utf-8-sig")
    else:
        raise ValueError("Test dataset must be .json or .csv")

    required_columns = ['text', 'image_path', 'label']
    if not all(column in test_data.columns for column in required_columns):
        raise ValueError(f"Test data must contain {required_columns}")

    # Load few-shot examples (only once if consistency=True)
    if few_shot_num > 0:
        few_shot_examples = load_few_shot_examples(devdata_path, few_shot_num)
        for example in few_shot_examples:
            if use_images:
                example["image"] = load_and_process_image(example["image_path"])
            else:
                example["image"] = None
    else:
        few_shot_examples = None

    results = []
    y_true_list = []
    y_pred_list = []
    total_tests = len(test_data)
    print(f"Total test cases: {total_tests}")

    start_time = time.time()

    for index, row in test_data.iterrows():
        image_path = row["image_path"]
        y_true = row["label"]
        text = row["text"]

        # If few-shot examples should vary per test case, reload new samples
        if (few_shot_num > 0 and not consistency):
            few_shot_examples = load_few_shot_examples(devdata_path, few_shot_num)
            for example in few_shot_examples:
                if use_images:
                    example["image"] = load_and_process_image(example["image_path"])

        y_pred = predict(
            model=model,
            processor=processor,
            prompt=prompt,
            task=task,
            use_texts=use_texts,
            use_images=use_images,
            text=text,
            image_path=image_path,
            few_shot_examples=few_shot_examples,
            few_shot_num=few_shot_num
        )

        # Time tracking at checkpoints
        if index == total_tests // 100:  # After 1% checkpoint
            update_remaining_time(start_time, index + 1, total_tests, "1% of tests")
        elif index == total_tests // 4:  # 25% checkpoint
            update_remaining_time(start_time, index + 1, total_tests, "25% of tests")
        elif index == total_tests // 2:  # 50% checkpoint
            update_remaining_time(start_time, index + 1, total_tests, "50% of tests")
        elif index == (3 * total_tests) // 4:  # 75% checkpoint
            update_remaining_time(start_time, index + 1, total_tests, "75% of tests")

        if y_pred is None:  # Ignore invalid results
            print(f"\nBad result! Ignore this test case and move on!\n")
            continue

        results.append({
            "image_path": image_path,
            "y_true": y_true,
            "y_pred": y_pred,
        })
        y_true_list.append(y_true)
        y_pred_list.append(y_pred)

    # Save results
    with open(result_path, "w", encoding="utf-8") as f:
        json.dump(results, f, indent=4, ensure_ascii=False)
    print(f"\nPerformed {len(results)} test cases. Predictions saved to {result_path}")

    # Calculate & display test statistics
    if y_true_list:
        accuracy = accuracy_score(y_true_list, y_pred_list)
        precision, recall, f1, _ = precision_recall_fscore_support(
            y_true_list,
            y_pred_list,
            average="weighted"
        )

        print("\n📊 **Test Statistics**")
        print(f"✔ Accuracy: {accuracy:.4f}")
        print(f"✔ Precision: {precision:.4f}")
        print(f"✔ Recall: {recall:.4f}")
        print(f"✔ F1 Score: {f1:.4f}")

        # Save statistics
        stats = {
            "accuracy": accuracy,
            "precision": precision,
            "recall": recall,
            "f1_score": f1
        }
        stats_path = result_path.replace(".json", "_stats.json")
        with open(stats_path, "w", encoding="utf-8") as f:
            json.dump(stats, f, indent=4, ensure_ascii=False)

        print(f"\n✅ Test statistics saved to {stats_path}")

    # Clear GPU memory
    with torch.no_grad():
        torch.cuda.empty_cache()

    return model

    # Optional memory cleanup
    with torch.no_grad():
        torch.cuda.empty_cache()  # Clear unused GPU memory after inference
        
            

In [3]:
# ensure to only use gpu 0 in the case that the other gpu has smaller VRAM than gpu 0
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

import torch
print(torch.cuda.device_count())  # Should print 1
print(torch.cuda.current_device())  # Should print 0

1
0


# INFORMATIVE - ZEROSHOT TEXT ONLY

In [3]:
model = test_model(
    model_id="meta-llama/Llama-3.2-11B-Vision-Instruct",
    task="Informative",
    use_texts=True,
    use_images=False,
    prompt=INFO_TEXT_ONLY_PROMPT,
    testdata_path="dataset/Informative/info_text_only_test.json",
    devdata_path="dataset/Informative/info_text_only_dev.json",
    result_path="test_results/Informative/Test-23/Llama-3.2-11B-Info-Zeroshot-Text-Only.json",
    few_shot_num=0,
    consistency=True
)

Model=meta-llama/Llama-3.2-11B-Vision-Instruct, testdata=dataset\Informative\info_text_only_test.json, result=test_results\Informative\Test-23\Llama-3.2-11B-Info-Zeroshot-Text-Only.json

Loading Llama-Vision model from meta-llama/Llama-3.2-11B-Vision-Instruct ...


The model weights are not tied. Please use the `tie_weights` method before using the `infer_auto_device` function.
Loading checkpoint shards: 100%|█████████████████████████████████████████████████████████| 5/5 [00:16<00:00,  3.40s/it]


Loading test dataset from dataset\Informative\info_text_only_test.json
Total test cases: 1534


 --------------------------------------------------------------------------------
[
  {
    "role": "user",
    "content": [
      {
        "type": "text",
        "text": " You are an AI model that classifies the given text into one of two categories based on its informational value. \nYour task is to analyze the given text to determine whether it contains relevant or informative content about a crisis.\nOur goal is to collect as much useful information as possible about crises, so your classification should prioritize identifying texts that provide relevant details, even if they are brief or incomplete. \nAvoid being overly restrictive. If the text has any relevant crisis-related information, classify it as informative.\nClassify the text delimited by triple quotes (\"\"\" \"\"\") into one of the following categories:\n  - **1 (positive):** The text provide details, updates, or any releva

C:\Users\anhtr\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\generation\configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
C:\Users\anhtr\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\generation\configuration_utils.py:633: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
C:\Users\anhtr\AppData\Local\Programs\Python\Python312\Lib\site-packages\PIL\Image.py:981: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(



📊 1% of tests completed. Estimated remaining time: 3.66 minutes.


📊 25% of tests completed. Estimated remaining time: 2.13 minutes.


📊 50% of tests completed. Estimated remaining time: 1.42 minutes.


📊 75% of tests completed. Estimated remaining time: 0.71 minutes.


Performed 1534 test cases. Predictions saved to test_results\Informative\Test-23\Llama-3.2-11B-Info-Zeroshot-Text-Only.json

📊 **Test Statistics**
✔ Accuracy: 0.8357
✔ Precision: 0.8328
✔ Recall: 0.8357
✔ F1 Score: 0.8322

✅ Test statistics saved to test_results\Informative\Test-23\Llama-3.2-11B-Info-Zeroshot-Text-Only_stats.json


# INFORMATIVE - ONESHOT TEXT ONLY

In [4]:
model = test_model(
    model_id="meta-llama/Llama-3.2-11B-Vision-Instruct",
    task="Informative",
    use_texts=True,
    use_images=False,
    prompt=INFO_TEXT_ONLY_PROMPT,
    testdata_path="dataset/Informative/info_text_only_test.json",
    devdata_path="dataset/Informative/info_text_only_dev.json",
    result_path="test_results/Informative/Test-23/Llama-3.2-11B-Info-Oneshot-Text-Only.json",
    few_shot_num=1,
    consistency=True
)

Model=meta-llama/Llama-3.2-11B-Vision-Instruct, testdata=dataset\Informative\info_text_only_test.json, result=test_results\Informative\Test-23\Llama-3.2-11B-Info-Oneshot-Text-Only.json

Loading Llama-Vision model from meta-llama/Llama-3.2-11B-Vision-Instruct ...


The model weights are not tied. Please use the `tie_weights` method before using the `infer_auto_device` function.
Loading checkpoint shards: 100%|█████████████████████████████████████████████████████████| 5/5 [00:08<00:00,  1.79s/it]


Loading test dataset from dataset\Informative\info_text_only_test.json
Total test cases: 1534


C:\Users\anhtr\AppData\Local\Temp\ipykernel_33152\2944525729.py:273: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  label_samples = dev_data.groupby("label").apply(lambda x: x.sample(n=min(few_shot_num, len(x)), random_state=42)).reset_index(drop=True)
C:\Users\anhtr\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\generation\configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
C:\Users\anhtr\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\generation\configuration_u


📊 1% of tests completed. Estimated remaining time: 2.91 minutes.


📊 25% of tests completed. Estimated remaining time: 2.20 minutes.


📊 50% of tests completed. Estimated remaining time: 1.48 minutes.


📊 75% of tests completed. Estimated remaining time: 0.74 minutes.


Performed 1534 test cases. Predictions saved to test_results\Informative\Test-23\Llama-3.2-11B-Info-Oneshot-Text-Only.json

📊 **Test Statistics**
✔ Accuracy: 0.8279
✔ Precision: 0.8446
✔ Recall: 0.8279
✔ F1 Score: 0.8122

✅ Test statistics saved to test_results\Informative\Test-23\Llama-3.2-11B-Info-Oneshot-Text-Only_stats.json


# INFORMATIVE - FIVESHOT TEXT ONLY

In [5]:
model = test_model(
    model_id="meta-llama/Llama-3.2-11B-Vision-Instruct",
    task="Informative",
    use_texts=True,
    use_images=False,
    prompt=INFO_TEXT_ONLY_PROMPT,
    testdata_path="dataset/Informative/info_text_only_test.json",
    devdata_path="dataset/Informative/info_text_only_dev.json",
    result_path="test_results/Informative/Test-23/Llama-3.2-11B-Info-Fiveshot-Text-Only.json",
    few_shot_num=5,
    consistency=True
)

Model=meta-llama/Llama-3.2-11B-Vision-Instruct, testdata=dataset\Informative\info_text_only_test.json, result=test_results\Informative\Test-23\Llama-3.2-11B-Info-Fiveshot-Text-Only.json

Loading Llama-Vision model from meta-llama/Llama-3.2-11B-Vision-Instruct ...


The model weights are not tied. Please use the `tie_weights` method before using the `infer_auto_device` function.
Loading checkpoint shards: 100%|█████████████████████████████████████████████████████████| 5/5 [00:02<00:00,  2.28it/s]
Some parameters are on the meta device because they were offloaded to the cpu.


Loading test dataset from dataset\Informative\info_text_only_test.json
Total test cases: 1534


C:\Users\anhtr\AppData\Local\Temp\ipykernel_33152\2944525729.py:273: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  label_samples = dev_data.groupby("label").apply(lambda x: x.sample(n=min(few_shot_num, len(x)), random_state=42)).reset_index(drop=True)
C:\Users\anhtr\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\generation\configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
C:\Users\anhtr\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\generation\configuration_u


📊 1% of tests completed. Estimated remaining time: 61.99 minutes.


📊 25% of tests completed. Estimated remaining time: 43.31 minutes.


📊 50% of tests completed. Estimated remaining time: 28.88 minutes.


📊 75% of tests completed. Estimated remaining time: 14.46 minutes.


Performed 1534 test cases. Predictions saved to test_results\Informative\Test-23\Llama-3.2-11B-Info-Fiveshot-Text-Only.json

📊 **Test Statistics**
✔ Accuracy: 0.8253
✔ Precision: 0.8382
✔ Recall: 0.8253
✔ F1 Score: 0.8104

✅ Test statistics saved to test_results\Informative\Test-23\Llama-3.2-11B-Info-Fiveshot-Text-Only_stats.json


# INFORMATIVE - ZEROSHOT IMAGE ONLY

In [5]:
model = test_model(
    model_id="meta-llama/Llama-3.2-11B-Vision-Instruct",
    task="Informative",
    use_texts=False,
    use_images=True,
    prompt=INFO_IMAGE_ONLY_PROMPT,
    testdata_path="dataset/Informative/info_image_only_test.json",
    devdata_path="dataset/Informative/info_image_only_dev.json",
    result_path="test_results/Informative/Test-23/Llama-3.2-11B-Info-Zeroshot-Image-Only.json",
    few_shot_num=0,
    consistency=True
)

Model=meta-llama/Llama-3.2-11B-Vision-Instruct, testdata=dataset\Informative\info_image_only_test.json, result=test_results\Informative\Test-23\Llama-3.2-11B-Info-Zeroshot-Image-Only.json

Loading Llama-Vision model from meta-llama/Llama-3.2-11B-Vision-Instruct ...


The model weights are not tied. Please use the `tie_weights` method before using the `infer_auto_device` function.
Loading checkpoint shards: 100%|█████████████████████████████████████████████████████████| 5/5 [00:09<00:00,  1.84s/it]


Loading test dataset from dataset\Informative\info_image_only_test.json
Total test cases: 1534


 --------------------------------------------------------------------------------
[
  {
    "role": "user",
    "content": [
      {
        "type": "image",
        "image": "<PIL.Image mode=RGB size=(1200, 799)>"
      },
      {
        "type": "text",
        "text": "Does the image give useful information that could help during a crisis?\nRespond with '1' if this image provides any information or details about a crisis, and '0' if it does not.\n\nInstructions: \n  - You should prioritize identifying texts that provide relevant details, even if they are brief or incomplete.\n  - Avoid being overly restrictive. If the text has any relevant crisis-related information, response with '1'.\n  - When the meaning of the image is unclear, response with '0'.\n  - Do not output any extra text.\n\nAbove is the test image that you need to classify. The category of this test image is: "
      }
    

C:\Users\anhtr\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\generation\configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
C:\Users\anhtr\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\generation\configuration_utils.py:633: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
C:\Users\anhtr\AppData\Local\Programs\Python\Python312\Lib\site-packages\PIL\Image.py:981: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(



📊 1% of tests completed. Estimated remaining time: 15.24 minutes.

Unclear model output: The image is a screenshot of a music player app on a mobile device, displaying the song "Party! Trying to analyze the output again...
Analyze completed! No matching label found! Assigned as not informative by default!
Unclear model output: The image shows a weather radar map with a red box around a specific area, indicating a severe weather! Trying to analyze the output again...
Analyze completed! No matching label found! Assigned as not informative by default!

📊 25% of tests completed. Estimated remaining time: 11.25 minutes.

Unclear model output: The image shows a woman doing yoga on the beach, which is not related to a crisis. Therefore! Trying to analyze the output again...
Analyze completed! No matching label found! Assigned as not informative by default!

📊 50% of tests completed. Estimated remaining time: 7.51 minutes.


📊 75% of tests completed. Estimated remaining time: 3.75 minutes.




# INFORMATIVE - ONESHOT IMAGE ONLY

In [6]:
model = test_model(
    model_id="meta-llama/Llama-3.2-11B-Vision-Instruct",
    task="Informative",
    use_texts=False,
    use_images=True,
    prompt=INFO_IMAGE_ONLY_PROMPT,
    testdata_path="dataset/Informative/info_image_only_test.json",
    devdata_path="dataset/Informative/info_image_only_dev.json",
    result_path="test_results/Informative/Test-23/Llama-3.2-11B-Info-Oneshot-Image-Only.json",
    few_shot_num=1,
    consistency=True
)

Model=meta-llama/Llama-3.2-11B-Vision-Instruct, testdata=dataset\Informative\info_image_only_test.json, result=test_results\Informative\Test-23\Llama-3.2-11B-Info-Oneshot-Image-Only.json

Loading Llama-Vision model from meta-llama/Llama-3.2-11B-Vision-Instruct ...


The model weights are not tied. Please use the `tie_weights` method before using the `infer_auto_device` function.
Loading checkpoint shards: 100%|█████████████████████████████████████████████████████████| 5/5 [00:09<00:00,  1.85s/it]


Loading test dataset from dataset\Informative\info_image_only_test.json
Total test cases: 1534


C:\Users\anhtr\AppData\Local\Temp\ipykernel_40496\2944525729.py:273: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  label_samples = dev_data.groupby("label").apply(lambda x: x.sample(n=min(few_shot_num, len(x)), random_state=42)).reset_index(drop=True)
C:\Users\anhtr\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\generation\configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
C:\Users\anhtr\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\generation\configuration_u


📊 1% of tests completed. Estimated remaining time: 36.80 minutes.

Unclear model output: The image shows a screenshot of a music player on a mobile device, with the song "Party People! Trying to analyze the output again...
Analyze completed! No matching label found! Assigned as not informative by default!
Unclear model output: The image shows a news article about the White House interior, with a headline that reads "Trump has! Trying to analyze the output again...
Analyze completed! No matching label found! Assigned as not informative by default!
Unclear model output: The image shows a weather radar map with a red box around the area of interest, indicating a severe! Trying to analyze the output again...
Analyze completed! No matching label found! Assigned as not informative by default!

📊 25% of tests completed. Estimated remaining time: 28.75 minutes.

Unclear model output: The image shows a woman doing yoga on the beach, with the ocean in the background. The image! Trying to analyz

# INFORMATIVE - ZEROSHOT TEXT + IMAGE

In [4]:
model = test_model(
    model_id="meta-llama/Llama-3.2-11B-Vision-Instruct",
    task="Informative",
    use_texts=True,
    use_images=True,
    prompt=INFO_TEXT_IMAGE_PROMPT,
    testdata_path="dataset/Informative/info_text_image_test.json",
    devdata_path="dataset/Informative/info_text_image_dev.json",
    result_path="test_results/Informative/Test-23/Llama-3.2-11B-Info-Zeroshot-Text-Image.json",
    few_shot_num=0,
    consistency=True
)

Model=meta-llama/Llama-3.2-11B-Vision-Instruct, testdata=dataset\Informative\info_text_image_test.json, result=test_results\Informative\Test-23\Llama-3.2-11B-Info-Zeroshot-Text-Image.json

Loading Llama-Vision model from meta-llama/Llama-3.2-11B-Vision-Instruct ...


The model weights are not tied. Please use the `tie_weights` method before using the `infer_auto_device` function.
Loading checkpoint shards: 100%|█████████████████████████████████████████████████████████| 5/5 [00:08<00:00,  1.74s/it]


Loading test dataset from dataset\Informative\info_text_image_test.json
Total test cases: 1534


 --------------------------------------------------------------------------------
[
  {
    "role": "user",
    "content": [
      {
        "type": "image",
        "image": "<PIL.Image mode=RGB size=(1200, 799)>"
      },
      {
        "type": "text",
        "text": "Do the given text and the given image give useful information that could help during a crisis?\nRespond with '1' if the text and the image provide any information or details about a crisis, and '0' if they do not.\n\nInstructions: \n  - You should prioritize identifying texts and the images that provide relevant details, even if they are brief or incomplete.\n  - Avoid being overly restrictive. If the text and image have any relevant crisis-related information, response with '1'.\n  - When the meaning of the image and the text are unclear, response with '0'.\n  - Do not output any extra text.\n\nAbove is the given image, a

C:\Users\anhtr\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\generation\configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
C:\Users\anhtr\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\generation\configuration_utils.py:633: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
C:\Users\anhtr\AppData\Local\Programs\Python\Python312\Lib\site-packages\PIL\Image.py:981: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(



📊 1% of tests completed. Estimated remaining time: 16.24 minutes.

Unclear model output: The given text and image do not provide any information that could help during a crisis.! Trying to analyze the output again...
Analyze successfully! This entry is not informative!
Unclear model output: The given text and image provide useful information that could help during a crisis. The text mentions a tornado! Trying to analyze the output again...
Analyze successfully! This entry is informative!

📊 25% of tests completed. Estimated remaining time: 11.94 minutes.

Unclear model output: The given text and image do not provide any information that could help during a crisis.! Trying to analyze the output again...
Analyze successfully! This entry is not informative!

📊 50% of tests completed. Estimated remaining time: 7.94 minutes.


📊 75% of tests completed. Estimated remaining time: 3.96 minutes.


Performed 1534 test cases. Predictions saved to test_results\Informative\Test-23\Llama-3.2-11B-In

# INFORMATIVE - ONESHOT TEXT + IMAGE

In [4]:
model = test_model(
    model_id="meta-llama/Llama-3.2-11B-Vision-Instruct",
    task="Informative",
    use_texts=True,
    use_images=True,
    prompt=INFO_TEXT_IMAGE_PROMPT,
    testdata_path="dataset/Informative/info_text_image_test.json",
    devdata_path="dataset/Informative/info_text_image_dev.json",
    result_path="test_results/Informative/Test-23/Llama-3.2-11B-Info-Oneshot-Text-Image.json",
    few_shot_num=1,
    consistency=True
)

Model=meta-llama/Llama-3.2-11B-Vision-Instruct, testdata=dataset\Informative\info_text_image_test.json, result=test_results\Informative\Test-23\Llama-3.2-11B-Info-Oneshot-Text-Image.json

Loading Llama-Vision model from meta-llama/Llama-3.2-11B-Vision-Instruct ...


The model weights are not tied. Please use the `tie_weights` method before using the `infer_auto_device` function.
Loading checkpoint shards: 100%|█████████████████████████████████████████████████████████| 5/5 [00:08<00:00,  1.74s/it]


Loading test dataset from dataset\Informative\info_text_image_test.json
Total test cases: 1534


 --------------------------------------------------------------------------------
[
  {
    "role": "user",
    "content": [
      {
        "type": "image",
        "image": "<PIL.Image mode=RGB size=(1000, 600)>"
      },
      {
        "type": "text",
        "text": "Above is the image, and here is the Text: @UofR welcomes displaced college students from Puerto Rico, U.S. Virgin Islands https://t.co/urXHYbB2Ea https://t.co/eWloa8TnIF\nThe category of this image and Text is: 0"
      },
      {
        "type": "image",
        "image": "<PIL.Image mode=RGB size=(960, 500)>"
      },
      {
        "type": "text",
        "text": "Above is the image, and here is the Text: Rebuilding Puerto Rico from the Grassroots Up https://t.co/iP2szQdZyO via @CityLab https://t.co/i7YBGLSJca\nThe category of this image and Text is: 1"
      },
      {
        "type": "image",
        "image": "<PIL.Im

C:\Users\anhtr\AppData\Local\Temp\ipykernel_47348\3544193586.py:273: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  label_samples = dev_data.groupby("label").apply(lambda x: x.sample(n=min(few_shot_num, len(x)), random_state=42)).reset_index(drop=True)
C:\Users\anhtr\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\generation\configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
C:\Users\anhtr\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\generation\configuration_u

Unclear model output: The given text and image do not provide any information that could help during a crisis.! Trying to analyze the output again...
Analyze successfully! This entry is not informative!
Unclear model output: The given text and image provide information about Hurricane Maria's track and its potential impact on the continental US! Trying to analyze the output again...
Analyze successfully! This entry is informative!
Unclear model output: The given text and image provide information about a crisis, specifically the location and movement of Tropical Storm Maria! Trying to analyze the output again...
Analyze successfully! This entry is informative!
Unclear model output: The given text and image provide information about a benefit concert to raise money for hurricane relief in Puerto Rico! Trying to analyze the output again...
Analyze successfully! This entry is informative!
Unclear model output: The given text and image do not provide any information that could help during 

C:\Users\anhtr\AppData\Local\Programs\Python\Python312\Lib\site-packages\PIL\Image.py:981: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


Unclear model output: The given text and image do not provide any information that could help during a crisis. The text appears! Trying to analyze the output again...
Analyze successfully! This entry is not informative!
Unclear model output: The given text and image do not provide any information that could help during a crisis. The text appears! Trying to analyze the output again...
Analyze successfully! This entry is not informative!

📊 1% of tests completed. Estimated remaining time: 65.60 minutes.

Unclear model output: The given text and image do not provide any information that could help during a crisis. The image appears! Trying to analyze the output again...
Analyze successfully! This entry is not informative!
Unclear model output: The given text and image provide information about a crisis, specifically Hurricane Irma, and its impact on St! Trying to analyze the output again...
Analyze successfully! This entry is informative!
Unclear model output: The given text and image do 

# HUMANITARIAN - ZEROSHOT TEXT + IMAGE ON FINETUNED TEXT+IMAGE

In [5]:
model = test_model(
    model_id="models/Llama-3.2-11B-Vision/humanitarian/text_image",
    task="Humanitarian",
    use_texts=True,
    use_images=True,
    prompt=HUMA_TEXT_IMAGE_PROMPT,
    testdata_path="dataset/Humanitarian/huma_text_image_test.json",
    devdata_path="dataset/Humanitarian/huma_text_image_dev.json",
    result_path="test_results/Humanitarian/Test-23/Llama-3.2-11B-TextImage-Huma-Zeroshot-Text-Image.json",
    few_shot_num=0,
    consistency=True
)

Model=models/Llama-3.2-11B-Vision/humanitarian/text_image, testdata=dataset\Humanitarian\huma_text_image_test.json, result=test_results\Humanitarian\Test-23\Llama-3.2-11B-TextImage-Huma-Zeroshot-Text-Image.json

Loading Llama-Vision model from models/Llama-3.2-11B-Vision/humanitarian/text_image ...


The model weights are not tied. Please use the `tie_weights` method before using the `infer_auto_device` function.
Loading checkpoint shards: 100%|█████████████████████████████████████████████████████████| 5/5 [00:47<00:00,  9.54s/it]


Loading test dataset from dataset\Humanitarian\huma_text_image_test.json
Total test cases: 955


D:\Research\LLM_FT\llm-env\Lib\site-packages\transformers\generation\configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
D:\Research\LLM_FT\llm-env\Lib\site-packages\transformers\generation\configuration_utils.py:633: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(



📊 1% of tests completed. Estimated remaining time: 26.94 minutes.


📊 25% of tests completed. Estimated remaining time: 20.48 minutes.


📊 50% of tests completed. Estimated remaining time: 13.65 minutes.


📊 75% of tests completed. Estimated remaining time: 6.82 minutes.


Performed 955 test cases. Predictions saved to test_results\Humanitarian\Test-23\Llama-3.2-11B-TextImage-Huma-Zeroshot-Text-Image.json

📊 **Test Statistics**
✔ Accuracy: 0.8723
✔ Precision: 0.8777
✔ Recall: 0.8723
✔ F1 Score: 0.8679

✅ Test statistics saved to test_results\Humanitarian\Test-23\Llama-3.2-11B-TextImage-Huma-Zeroshot-Text-Image_stats.json


# HUMANITARIAN - ONESHOT TEXT + IMAGE ON FINETUNED TEXT+IMAGE

In [4]:
model = test_model(
    model_id="models/Llama-3.2-11B-Vision/humanitarian/text_image",
    task="Humanitarian",
    use_texts=True,
    use_images=True,
    prompt=HUMA_TEXT_IMAGE_PROMPT,
    testdata_path="dataset/Humanitarian/huma_text_image_test.json",
    devdata_path="dataset/Humanitarian/huma_text_image_dev.json",
    result_path="test_results/Humanitarian/Test-23/Llama-3.2-11B-TextImage-Huma-Oneshot-Text-Image.json",
    few_shot_num=1,
    consistency=True
)

Model=models/Llama-3.2-11B-Vision/humanitarian/text_image, testdata=dataset\Humanitarian\huma_text_image_test.json, result=test_results\Humanitarian\Test-23\Llama-3.2-11B-TextImage-Huma-Oneshot-Text-Image.json

Loading Llama-Vision model from models/Llama-3.2-11B-Vision/humanitarian/text_image ...


The model weights are not tied. Please use the `tie_weights` method before using the `infer_auto_device` function.
Loading checkpoint shards: 100%|█████████████████████████████████████████████████████████| 5/5 [00:40<00:00,  8.11s/it]


Loading test dataset from dataset\Humanitarian\huma_text_image_test.json


C:\Users\anhtr\AppData\Local\Temp\ipykernel_70948\2944525729.py:273: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  label_samples = dev_data.groupby("label").apply(lambda x: x.sample(n=min(few_shot_num, len(x)), random_state=42)).reset_index(drop=True)


Total test cases: 955


 --------------------------------------------------------------------------------
[
  {
    "role": "user",
    "content": [
      {
        "type": "image",
        "image": "<PIL.Image mode=RGB size=(768, 1024)>"
      },
      {
        "type": "text",
        "text": "Above is the sample image, and here is the Text: \"\"\"Flood #SriLanka. Lady explained how she saved family lives but lost house. @NorwayMFA provides assist of USD1.2 mill https://t.co/oQDBXp0en7\"\"\"\nThe category of this sample text and image is: 0"
      },
      {
        "type": "image",
        "image": "<PIL.Image mode=RGB size=(770, 433)>"
      },
      {
        "type": "text",
        "text": "Above is the sample image, and here is the Text: \"\"\"At least $31,000 raised in MU-UWM exhibition game for Puerto Rico hurricane relief https://t.co/xPak8vZ9wB https://t.co/yRruJ51rjN\"\"\"\nThe category of this sample text and image is: 1"
      },
      {
        "type": "image",
        "

D:\Research\LLM_FT\llm-env\Lib\site-packages\transformers\generation\configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
D:\Research\LLM_FT\llm-env\Lib\site-packages\transformers\generation\configuration_utils.py:633: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(



📊 1% of tests completed. Estimated remaining time: 93.93 minutes.


📊 25% of tests completed. Estimated remaining time: 70.98 minutes.


📊 50% of tests completed. Estimated remaining time: 47.23 minutes.


📊 75% of tests completed. Estimated remaining time: 23.56 minutes.


Performed 955 test cases. Predictions saved to test_results\Humanitarian\Test-23\Llama-3.2-11B-TextImage-Huma-Oneshot-Text-Image.json

📊 **Test Statistics**
✔ Accuracy: 0.8209
✔ Precision: 0.8303
✔ Recall: 0.8209
✔ F1 Score: 0.8047

✅ Test statistics saved to test_results\Humanitarian\Test-23\Llama-3.2-11B-TextImage-Huma-Oneshot-Text-Image_stats.json


D:\Research\LLM_FT\llm-env\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
